**Check Python version. This notebook is implemented for Python 3.6.x. Not all cells may work in other versions of Python.**

In [1]:
import platform
import os
print(platform.python_version())


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191204194951-0000
KERNEL_ID = d3b5afbb-2908-4ee8-aeb2-3b0dd46f800b
3.6.8


### Predicting Customer Churn in Telco

In this notebook you will learn how to build a predictive model with Spark machine learning API (SparkML) and deploy it for scoring in Machine Learning (ML). 

This notebook walks you through these steps:
- Build a model with SparkML API
- Save the model in the ML repository
- Create a Deployment in ML (via UI)
- Test the model (via UI)
- Test the model (via REST API)

### Step 1: Review Use Case

The analytics use case implemented in this notebook is telco churn. While it's a simple use case, it implements all steps from the CRISP-DM methodolody, which is the recommended best practice for implementing predictive analytics. 
![CRISP-DM](https://raw.githubusercontent.com/yfphoon/dsx_demo/master/crisp_dm.png)

The analytics process starts with defining the business problem and identifying the data that can be used to solve the problem. For Telco churn, we use demographic and historical transaction data. We also know which customers have churned, which is the critical information for building predictive models. In the next step, we use visual APIs for data understanding and complete some data preparation tasks. In a typical analytics project data preparation will include more steps (for example, formatting data or deriving new variables). 

Once the data is ready, we can build a predictive model. In our example we are using the SparkML Random Forrest classification model. Classification is a statistical technique which assigns a "class" to each customer record (for our use case "churn" or "no churn"). Classification models use historical data to come up with the logic to predict "class", this process is called model training. After the model is created, it's usually evaluated using another data set. 

Finally, if the model's accuracy meets the expectations, it can be deployed for scoring. Scoring is the process of applying the model to a new set of data. For example, when we receive new transactional data, we can score the customer for the risk of churn.  

We also developed a sample Python Flask application to illustrate deployment: http://predictcustomerchurn.mybluemix.net/. This application implements the REST client call to the model.

### Working with Notebooks

If you are new to Notebooks, here's a quick overview of how to work in this environment.

1. The notebook has 2 types of cells - markdown (text) and code. 
2. Each cell with code can be executed independently or together (see options under the Cell menu). When working in this notebook, we will be running one cell at a time because we need to make code changes to some of the cells.
3. To run the cell, position cursor in the code cell and click the Run (arrow) icon. The cell is running when you see the * next to it. Some cells have printable output.
4. Work through this notebook by reading the instructions and executing code cell by cell. Some cells will require modifications before you run them. 

### Step 2: Load data 

In [2]:
# Customer Information
customer = spark.read.format("csv").load('/project_data/data_asset/customer.csv', header='true', inferSchema = 'true')
customer.show(5)
#Churn information    
customer_churn = spark.read.format("csv").load('/project_data/data_asset/churn.csv', header='true', inferSchema = 'true')
# churn.show(5)
customer.take(5)

+---+------+------+--------+----------+---------+---------+------------+-------------+------+-------+---------+-------------+--------------------+------+--------+
| ID|Gender|Status|Children|Est Income|Car Owner|      Age|LongDistance|International| Local|Dropped|Paymethod|LocalBilltype|LongDistanceBilltype| Usage|RatePlan|
+---+------+------+--------+----------+---------+---------+------------+-------------+------+-------+---------+-------------+--------------------+------+--------+
|  1|     F|     S|     1.0|   38000.0|        N|24.393333|       23.56|          0.0|206.08|    0.0|       CC|       Budget|      Intnl_discount|229.64|     3.0|
|  6|     M|     M|     2.0|   29616.0|        N|49.426667|       29.78|          0.0|  45.5|    0.0|       CH|    FreeLocal|            Standard| 75.29|     2.0|
|  8|     M|     M|     0.0|   19732.8|        N|50.673333|       24.81|          0.0| 22.44|    0.0|       CC|    FreeLocal|            Standard| 47.25|     3.0|
| 11|     M|     S|   

[Row(ID=1, Gender='F', Status='S', Children=1.0, Est Income=38000.0, Car Owner='N', Age=24.393333, LongDistance=23.56, International=0.0, Local=206.08, Dropped=0.0, Paymethod='CC', LocalBilltype='Budget', LongDistanceBilltype='Intnl_discount', Usage=229.64, RatePlan=3.0),
 Row(ID=6, Gender='M', Status='M', Children=2.0, Est Income=29616.0, Car Owner='N', Age=49.426667, LongDistance=29.78, International=0.0, Local=45.5, Dropped=0.0, Paymethod='CH', LocalBilltype='FreeLocal', LongDistanceBilltype='Standard', Usage=75.29, RatePlan=2.0),
 Row(ID=8, Gender='M', Status='M', Children=0.0, Est Income=19732.8, Car Owner='N', Age=50.673333, LongDistance=24.81, International=0.0, Local=22.44, Dropped=0.0, Paymethod='CC', LocalBilltype='FreeLocal', LongDistanceBilltype='Standard', Usage=47.25, RatePlan=3.0),
 Row(ID=11, Gender='M', Status='S', Children=2.0, Est Income=96.33, Car Owner='N', Age=56.473333, LongDistance=26.13, International=0.0, Local=32.88, Dropped=1.0, Paymethod='CC', LocalBilltype

If the first step ran successfully (you saw the output), then continue reviewing the notebook and running each code cell step by step. Note that not every cell has a visual output. The cell is still running if you see a * in the brackets next to the cell. 

If the first step didn't finish successfully, check with the instructor. 

### Step 3: Merge Files

In [3]:
data=customer.join(customer_churn,customer['ID']==customer_churn['ID']).select(customer['*'],customer_churn['CHURN'])


### Step 4: Rename some columns
This step is to remove spaces from columns names, it's an example of data preparation that you may have to do before creating a model. 

In [4]:
data = data.withColumnRenamed("Est Income", "EstIncome").withColumnRenamed("Car Owner","CarOwner")
data.toPandas().head()

,ID,Gender,Status,Children,EstIncome,CarOwner,Age,LongDistance,International,Local,Dropped,Paymethod,LocalBilltype,LongDistanceBilltype,Usage,RatePlan,CHURN
0,1,F,S,1.0,38000.00,N,24.393333,23.56,0.0,206.08,0.0,CC,Budget,Intnl_discount,229.64,3.0,T
1,6,M,M,2.0,29616.00,N,49.426667,29.78,0.0,45.50,0.0,CH,FreeLocal,Standard,75.29,2.0,F
2,8,M,M,0.0,19732.80,N,50.673333,24.81,0.0,22.44,0.0,CC,FreeLocal,Standard,47.25,3.0,F
3,11,M,S,2.0,96.33,N,56.473333,26.13,0.0,32.88,1.0,CC,Budget,Standard,59.01,1.0,F
4,14,F,M,2.0,52004.80,N,25.140000,5.03,0.0,23.11,0.0,CH,Budget,Intnl_discount,28.14,1.0,F


### Step 5: Data understanding

Data preparation and data understanding are the most time-consuming tasks in the data mining process. The data scientist needs to review and evaluate the quality of data before modeling.

Visualization is one of the ways to reivew data.

The Brunel Visualization Language is a highly succinct and novel language that defines interactive data visualizations based on tabular data. The language is well suited for both data scientists and business users. 
More information about Brunel Visualization: https://github.com/Brunel-Visualization/Brunel/wiki

Try Brunel visualization here: http://brunel.mybluemix.net/gallery_app/renderer

In [5]:
import brunel
df = data.toPandas()
%brunel data('df') bar x(CHURN) y(EstIncome) mean(EstIncome) color(LocalBilltype) stack tooltip(EstIncome) | x(LongDistance) y(Usage) point color(Paymethod) tooltip(LongDistance, Usage) :: width=1100, height=400 

<IPython.core.display.Javascript object>

**PixieDust** is a Python Helper library for Spark IPython Notebooks. One of it's main features are visualizations. You'll notice that unlike other APIs which produce just output, PixieDust creates an **interactive UI** in which you can explore data.

More information about PixieDust: https://github.com/ibm-cds-labs/pixiedust?cm_mc_uid=78151411419314871783930&cm_mc_sid_50200000=1487962969

In [6]:
!pip install --user --upgrade pixiedust

    100% |################################| 204kB 2.5MB/s eta 0:00:01
    100% |################################| 798kB 1.8MB/s eta 0:00:01
    100% |################################| 5.8MB 487kB/s eta 0:00:01
    100% |################################| 92kB 3.7MB/s eta 0:00:01
    100% |################################| 61kB 969kB/s eta 0:00:01
    100% |################################| 583kB 2.3MB/s eta 0:00:01
    100% |################################| 143kB 3.0MB/s eta 0:00:01
    100% |################################| 133kB 4.9MB/s eta 0:00:01
    100% |################################| 163kB 759kB/s eta 0:00:01
    100% |################################| 61kB 1.9MB/s eta 0:00:01
  Stored in directory: /home/spark/shared/.cache/pip/wheels/e8/b1/86/c2f2e16e6bf9bfe556f9dbf8adb9f41816c476d73078c7d0eb
  Stored in directory: /home/spark/shared/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
Successfully built pixiedust mpld3
tensorflow 1.13.1 requires t

In [ ]:
from pixiedust.display import *
display(data)

### Step 6: Build the Spark pipeline and the Random Forest model
"Pipeline" is an API in SparkML that's used for building models.
Additional information on SparkML: https://spark.apache.org/docs/2.0.2/ml-guide.html

In [8]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer, IndexToString
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# Prepare string variables so that they can be used by the decision tree algorithm
# StringIndexer encodes a string column of labels to a column of label indices
SI1 = StringIndexer(inputCol='Gender', outputCol='GenderEncoded')
SI2 = StringIndexer(inputCol='Status',outputCol='StatusEncoded')
SI3 = StringIndexer(inputCol='CarOwner',outputCol='CarOwnerEncoded')
SI4 = StringIndexer(inputCol='Paymethod',outputCol='PaymethodEncoded')
SI5 = StringIndexer(inputCol='LocalBilltype',outputCol='LocalBilltypeEncoded')
SI6 = StringIndexer(inputCol='LongDistanceBilltype',outputCol='LongDistanceBilltypeEncoded')
labelIndexer = StringIndexer(inputCol='CHURN', outputCol='label').fit(data)

# Pipelines API requires that input variables are passed in  a vector
assembler = VectorAssembler(inputCols=["GenderEncoded", "StatusEncoded", "CarOwnerEncoded", "PaymethodEncoded", "LocalBilltypeEncoded", \
                                       "LongDistanceBilltypeEncoded", "Children", "EstIncome", "Age", "LongDistance", "International", "Local",\
                                      "Dropped","Usage","RatePlan"], outputCol="features")

In [9]:
# instantiate the algorithm, take the default settings
rf=RandomForestClassifier(labelCol="label", featuresCol="features", impurity="gini")

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

pipeline = Pipeline(stages=[SI1,SI2,SI3,SI4,SI5,SI6,labelIndexer, assembler, rf, labelConverter])

In [10]:
# Split data into train and test datasets
train, test = data.randomSplit([0.8,0.2], seed=6)
train.cache()
test.cache()


DataFrame[ID: int, Gender: string, Status: string, Children: double, EstIncome: double, CarOwner: string, Age: double, LongDistance: double, International: double, Local: double, Dropped: double, Paymethod: string, LocalBilltype: string, LongDistanceBilltype: string, Usage: double, RatePlan: double, CHURN: string]

In [11]:
# Build models
model = pipeline.fit(train)


In [12]:
model.transform(test)


DataFrame[ID: int, Gender: string, Status: string, Children: double, EstIncome: double, CarOwner: string, Age: double, LongDistance: double, International: double, Local: double, Dropped: double, Paymethod: string, LocalBilltype: string, LongDistanceBilltype: string, Usage: double, RatePlan: double, CHURN: string, GenderEncoded: double, StatusEncoded: double, CarOwnerEncoded: double, PaymethodEncoded: double, LocalBilltypeEncoded: double, LongDistanceBilltypeEncoded: double, label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double, predictedLabel: string]

### Step 7: Score the test data set

In [13]:
results = model.transform(test)
results=results.select(results["ID"],results["CHURN"],results["label"],results["predictedLabel"],results["prediction"],results["probability"])
results.toPandas().head(6)


,ID,CHURN,label,predictedLabel,prediction,probability
0,14,F,0.0,F,0.0,"[0.8831540242807258, 0.11684597571927424]"
1,18,F,0.0,F,0.0,"[0.5864899484477009, 0.4135100515522991]"
2,21,F,0.0,F,0.0,"[0.7191228984151969, 0.28087710158480306]"
3,22,F,0.0,F,0.0,"[0.5851351347449855, 0.41486486525501454]"
4,29,T,1.0,T,1.0,"[0.17341532437349524, 0.8265846756265048]"
5,40,T,1.0,T,1.0,"[0.22284247048354122, 0.7771575295164588]"


### Step 8: Model Evaluation 

In [14]:
print('Precision model1 = {:.2f}.'.format(results.filter(results.label == results.prediction).count() / float(results.count())))


Precision model1 = 0.93.


In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")
print('Area under ROC curve = {:.2f}.'.format(evaluator.evaluate(results)))


Area under ROC curve = 0.92.


We have finished building and testing a predictive model. The next step is to deploy it for real time scoring. 

### Step 9: Save Model in ML repository

### Initializing WML Client

In [16]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import os

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "2.5.0"
}

client = WatsonMachineLearningAPIClient(wml_credentials)

### Creating the Deployment Space to save model into

In [17]:
# Creating space to deploy model into
"""
space_name = 'AlexbarDS'

space_meta_data = {
        client.spaces.ConfigurationMetaNames.NAME : space_name,
        client.spaces.ConfigurationMetaNames.TAGS : [{'value': 'teclco_churn_model_space'}]
}

stored_space_details = client.spaces.store(space_meta_data)

space_uid = stored_space_details['metadata']['guid']

client.set.default_space(space_uid)
"""

"\nspace_name = 'AlexbarDS'\n\nspace_meta_data = {\n        client.spaces.ConfigurationMetaNames.NAME : space_name,\n        client.spaces.ConfigurationMetaNames.TAGS : [{'value': 'teclco_churn_model_space'}]\n}\n\nstored_space_details = client.spaces.store(space_meta_data)\n\nspace_uid = stored_space_details['metadata']['guid']\n\nclient.set.default_space(space_uid)\n"

In [18]:
#space_uid

In [20]:
def guid_from_space_name(client, space_name):
   instance_details = client.service_instance.get_details()
   space = client.spaces.get_details()
   return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['guid'])
# Enter the name of your deployment space here:
space_uid = guid_from_space_name(client, 'alexbarDS')
print("Space UID = " + space_uid)

Space UID = 44dd2d4a-62ae-490e-a624-afb73cd568d4


In [21]:
space_uid

'44dd2d4a-62ae-490e-a624-afb73cd568d4'

### Save the pipeline model

In [22]:
model_name = 'TelcoChurn_model'

metadata = {
    client.repository.ModelMetaNames.NAME: model_name,
    client.repository.ModelMetaNames.TYPE: "mllib_2.3",
    client.repository.ModelMetaNames.RUNTIME_UID: "spark-mllib_2.3",
    client.repository.ModelMetaNames.TAGS: [{'value' : model_name}],
    client.repository.ModelMetaNames.SPACE_UID: space_uid
}

In [23]:
client.set.default_space(space_uid)

'SUCCESS'

In [24]:
metadata

{'name': 'TelcoChurn_model',
 'type': 'mllib_2.3',
 'runtime': 'spark-mllib_2.3',
 'tags': [{'value': 'TelcoChurn_model'}],
 'space': '44dd2d4a-62ae-490e-a624-afb73cd568d4'}

In [25]:
published_model_details = client.repository.store_model(model=model, meta_props=metadata, training_data=train, pipeline=pipeline)

In [26]:
model_uid = client.repository.get_model_uid(published_model_details)
print(model_uid)

22428b88-9daa-4dc3-ab0d-70b5c807eef4


In [27]:
client.spaces.list()

------------------------------------  ---------------------------  ------------------------
GUID                                  NAME                         CREATED
44dd2d4a-62ae-490e-a624-afb73cd568d4  alexbarDS                    2019-12-04T15:50:32.882Z
edc28d10-3e5c-46d0-8f41-75e1be7bdd98  srp - deployment space       2019-12-04T12:41:50.272Z
17bcf451-9364-4f9a-bcc6-a5036ef308fd  NA Test                      2019-12-02T21:09:24.250Z
9c6fd324-611d-4953-b2f5-b1f0010eebd4  checkpyv4                    2019-12-02T14:14:16.460Z
42168d32-883b-4aca-9aab-263f6ca7aaa4  IT_CHURN                     2019-12-01T14:34:46.068Z
2e3740c9-af63-400a-916e-0296f8156a68  customer_segmentation_space  2019-11-29T13:02:11.805Z
cd7a77ea-7e3a-4405-aac9-29506152a925  Dep Space                    2019-11-28T22:57:29.177Z
61205bd3-5b23-425f-980a-9e522d00ecfb  test                         2019-11-28T22:39:34.236Z
b90ec3f9-d32b-4244-ad60-b76f6ca81db2  customer_segmentation_space  2019-11-28T14:23:03.907Z
c1cb3

### Enabling (Making Online) the deployed model

In [28]:
deployment_name = 'telco-churn-model-deployment'

meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: deployment_name,
    client.deployments.ConfigurationMetaNames.TAGS : [{'value' : 'telcochurn_deployment_tag'}],
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}
#client.set.default_space(space_uid)
# Enabling the deployed model

deployment_details = client.deployments.create(artifact_uid=model_uid, meta_props=meta_props)



#######################################################################################

Synchronous deployment creation for uid: '22428b88-9daa-4dc3-ab0d-70b5c807eef4' started

#######################################################################################


initializing.
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='34d66604-e125-414f-958b-75579c770a83'
------------------------------------------------------------------------------------------------




In [29]:
scoring_url = client.deployments.get_scoring_href(deployment_details)
print ('Scoring Endpoint URL: '+ scoring_url)

# Fetching Deployment ID
deployment_id = client.deployments.get_uid(deployment_details)

Scoring Endpoint URL: https://internal-nginx-svc:12443/v4/deployments/34d66604-e125-414f-958b-75579c770a83/predictions


### Test model with a REST API cal

In [30]:
scoring_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{'fields': ['ID','Gender','Status','Children','EstIncome','CarOwner','Age','LongDistance',
                                                                'International','Local','Dropped','Paymethod','LocalBilltype','LongDistanceBilltype',
                                                                'Usage','RatePlan'],
                                                       'values': [[999,'F','M',2.0,77551.100000,'Y',33,20.530000,0.000000,41.890000,1.000000,'CC',
                                                                   'Budget','Standard',62.420000,2.000000]]
                                                     }]
}

client.deployments.score(deployment_id, scoring_payload)

{'predictions': [{'fields': ['ID',
    'Gender',
    'Status',
    'Children',
    'EstIncome',
    'CarOwner',
    'Age',
    'LongDistance',
    'International',
    'Local',
    'Dropped',
    'Paymethod',
    'LocalBilltype',
    'LongDistanceBilltype',
    'Usage',
    'RatePlan',
    'GenderEncoded',
    'StatusEncoded',
    'CarOwnerEncoded',
    'PaymethodEncoded',
    'LocalBilltypeEncoded',
    'LongDistanceBilltypeEncoded',
    'features',
    'rawPrediction',
    'probability',
    'prediction',
    'predictedLabel'],
   'values': [[999,
     'F',
     'M',
     2.0,
     77551.1,
     'Y',
     33.0,
     20.53,
     0.0,
     41.89,
     1.0,
     'CC',
     'Budget',
     'Standard',
     62.42,
     2.0,
     0.0,
     0.0,
     1.0,
     0.0,
     0.0,
     0.0,
     [0.0,
      0.0,
      1.0,
      0.0,
      0.0,
      0.0,
      2.0,
      77551.1,
      33.0,
      20.53,
      0.0,
      41.89,
      1.0,
      62.42,
      2.0],
     [18.266235498822613, 1.73376

In [ ]:
# Write the test data to a .csv so that we can later use it for Evaluation
writeCSV=test.toPandas()
writeCSV.to_csv('/project_data/data_asset/TelcoModelEval.csv', sep=',', index=False)


### Summary

You have finished working on this hands-on lab. In this notebook you created a model using SparkML API, deployed it in  Machine Learning service for online (real time) scoring and tested it using a test client. 


Created by **Sidney Phoon** and **Elena Lowery**
<br/>
yfphoon@us.ibm.com
elowery@us.ibm.com
<br/>
Jan 2, 2018